In [1]:
import sys
sys.path.append("../")
import os 
import glob 
import sklearn
import numpy as np
from PIL import Image
import torch
import torchvision
import torchvision.transforms as transforms
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
# from dataset import TinyImageNetDataset
from torch.utils.data import DataLoader
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import random
from torch.utils.data import DataLoader, Dataset, Subset
import medmnist
from medmnist import DermaMNIST, PathMNIST, BloodMNIST
random.seed(42)
transform = transforms.Compose([
    transforms.ToTensor()
])

## CIFAR10

In [ ]:
# Download and load training dataset
folder = '/local/home/abizeul/data/cifar-10-batches-py'
trainset = torchvision.datasets.CIFAR10(root='/local/home/abizeul/data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=len(trainset), shuffle=False)

# Fetch the entire dataset in one go
data_iter = iter(trainloader)
images, labels = next(data_iter)

# Step 2: Convert the dataset to NumPy arrays
images_np = images.numpy()
labels_np = labels.numpy()

# Reshape the images to (num_samples, height * width * channels)
num_samples = images_np.shape[0]
original_shape = images_np.shape
images_flat = images_np.reshape(num_samples, -1)

# Standardize
mean, std   = np.mean(images_flat, axis=0), np.std(images_flat, axis=0)
images_flat = (images_flat - mean) / std

# Step 4: Perform PCA
pca = PCA()  # You can adjust the number of components
pca.fit(images_flat)

In [ ]:
np.save(f'{folder}/pc_matrix.npy',pca.components_)
np.save(f'{folder}/eigenvalues.npy',pca.explained_variance_)
np.save(f'{folder}/eigenvalues_ratio.npy',pca.explained_variance_ratio_)

In [ ]:
# looking at eigenvalue decomposition 
# looking at eigenvalue decomposition 
fig = make_subplots(rows=1, cols=3, subplot_titles=("Eigenvalues", "Percentage explained variance","Cumulative sum of percentages"))

# First plot (left subplot)
fig.add_trace(go.Bar(
    x=np.arange(len(pca.explained_variance_)), 
    y=pca.explained_variance_,
    marker=dict(color='rgba(58, 71, 80, 0.6)', line=dict(color='rgba(58, 71, 80, 1.0)', width=1.5)),
    opacity=0.8
), row=1, col=1)

# Second plot (right subplot)
fig.add_trace(go.Bar(
    x=np.arange(len(pca.explained_variance_ratio_)), 
    y=pca.explained_variance_ratio_,  # Replace with a different dataset or transformation for a different plot
    marker=dict(color='rgba(120, 50, 80, 0.6)', line=dict(color='rgba(120, 50, 80, 1.0)', width=1.5)),
    opacity=0.8
), row=1, col=2)

fig.add_trace(go.Bar(
    x=np.arange(len(pca.explained_variance_ratio_)), 
    y=np.cumsum(pca.explained_variance_ratio_),  # Replace with a different dataset or transformation for a different plot
    marker=dict(color='rgba(120, 50, 80, 0.6)', line=dict(color='rgba(120, 50, 80, 1.0)', width=1.5)),
    opacity=0.8
), row=1, col=3)

# Update layout for aesthetics and specify the size
fig.update_layout(
    title_text='',
    xaxis=dict(
        title='',
        titlefont=dict(size=14),
        tickfont=dict(size=12),
        showgrid=True,  # Add grid lines for y-axis
        gridcolor='lightgrey',  # Color of grid lines
        gridwidth=0.5  # Width of grid lines
    ),
    yaxis=dict(
        title='',
        titlefont=dict(size=14),
        tickfont=dict(size=12),
        type='log',
        showgrid=True,  # Add grid lines for y-axis
        gridcolor='lightgrey',  # Color of grid lines
        gridwidth=0.5  # Width of grid lines
    ),
    plot_bgcolor='white',
    showlegend=False,
    margin=dict(l=40, r=40, t=40, b=40),
    width=1000,  # Specify width of the image
    height=600  # Specify height of the image
)

# Update x-axis and y-axis for the first subplot
fig.update_xaxes(title_text='Components', title_font=dict(size=14), tickfont=dict(size=12), 
                 showgrid=True, gridcolor='lightgrey', gridwidth=0.5, row=1, col=1)
fig.update_yaxes(title_text='Explained variance (Log Scale)', title_font=dict(size=14), tickfont=dict(size=12), 
                 type='log', showgrid=True, gridcolor='lightgrey', gridwidth=0.5, row=1, col=1)

# Update x-axis and y-axis for the second subplot
fig.update_xaxes(title_text='Components', title_font=dict(size=14), tickfont=dict(size=12), 
                 showgrid=True, gridcolor='lightgrey', gridwidth=0.5, row=1, col=2)
fig.update_yaxes(title_text='Explained variance (Log Scale)', title_font=dict(size=14), tickfont=dict(size=12), 
                 type='log', showgrid=True, gridcolor='lightgrey', gridwidth=0.5, row=1, col=2)

# Update x-axis and y-axis for the second subplot
fig.update_xaxes(title_text='Components', title_font=dict(size=14), tickfont=dict(size=12), 
                 showgrid=True, gridcolor='lightgrey', gridwidth=0.5, row=1, col=3)
fig.update_yaxes(title_text='Explained variance (Log Scale)', title_font=dict(size=14), tickfont=dict(size=12), 
                 type='log', showgrid=True, gridcolor='lightgrey', gridwidth=0.5, row=1, col=3)


# Save the figure as a PNG file
fig.write_image(f"{folder}/eigenvalues.png")

# Show the plot
fig.show()

In [ ]:
# Step 5: Visualize the result of pc filtering on one image
def reconstruct_image(pca, components, mean, std_mean, std_stdev, shape):
    image_reconstructed = np.dot(components, pca.components_) + mean
    image_reconstructed = (image_reconstructed * std_stdev) + std_mean
    image_reconstructed = image_reconstructed.reshape(shape)
    return image_reconstructed

ratios = [0.1,0.2,0.4,0.6,0.7,0.75,0.8,0.85,0.9,0.95,0.99]
nb_plots = len(ratios) + 1
nb_plots_row = 6
nb_components = min(images_flat.shape[0],images_flat.shape[1])
nb_plots_columns = int(np.ceil(nb_plots/nb_plots_row))

for j, img in enumerate(images_np[:2]):
    print(img.shape)
    fig, ax = plt.subplots(nb_plots_columns,nb_plots_row,figsize=(20,5))
    ax[0,0].imshow(np.transpose(img, (1,2,0)),cmap="gray")
    ax[0,0].set_title("Original Image")
    ax[0,0].axis('off')

    fig_, ax_ = plt.subplots(nb_plots_columns,nb_plots_row,figsize=(20,5))
    ax_[0,0].imshow(np.transpose(img, (1,2,0)),cmap="gray")
    ax_[0,0].set_title("Original Image")
    ax_[0,0].axis('off')

    fig2, ax2 = plt.subplots(nb_plots_columns,nb_plots_row,figsize=(20,5))
    ax2[0,0].imshow(np.transpose(img, (1,2,0)),cmap="gray")
    ax2[0,0].set_title("Original Image")
    ax2[0,0].axis('off')

    fig2_, ax2_ = plt.subplots(nb_plots_columns,nb_plots_row,figsize=(20,5))
    ax2_[0,0].imshow(np.transpose(img, (1,2,0)),cmap="gray")
    ax2_[0,0].set_title("Original Image")
    ax2_[0,0].axis('off')

    c=pca.transform(images_flat[j:j+1])
    cumsum = np.cumsum(pca.explained_variance_ratio_)

    for i, r in enumerate(ratios):
        row, column = (i+1)//nb_plots_row, (i+1)%nb_plots_row

        final_component = int(r*nb_components)
        components = np.zeros_like(c)
        components[:, :final_component] = c[:, :final_component]
        reconstructed_image = reconstruct_image(pca, components, pca.mean_, mean, std, [1]+list(original_shape[1:]))

        ax[row,column].imshow(np.transpose(reconstructed_image[0], (1,2,0)),cmap="gray")
        ax[row,column].set_title(f"ratio {r}")
        ax[row,column].axis('off')
        
        #######
        components = np.zeros_like(c)
        final_component = nb_components - final_component
        components[:, -final_component:] = c[:, -final_component:]
        reconstructed_image = reconstruct_image(pca, components, pca.mean_, mean, std, [1]+list(original_shape[1:]))

        ax_[row,column].imshow(np.transpose(reconstructed_image[0], (1,2,0)),cmap="gray")
        ax_[row,column].set_title(f"ratio {round(1-r,2)}")
        ax_[row,column].axis('off')

        #######
        final_component=int(np.argmin(np.abs(cumsum - r)))+1
        components = np.zeros_like(c)
        components[:, :final_component] = c[:, :final_component]
        reconstructed_image = reconstruct_image(pca, components, pca.mean_, mean, std, [1]+list(original_shape[1:]))

        ax2[row,column].imshow(np.transpose(reconstructed_image[0], (1,2,0)),cmap="gray")
        ax2[row,column].set_title(f"ratio {r}")
        ax2[row,column].axis('off')

        #######
        final_component = nb_components - final_component
        components = np.zeros_like(c)
        components[:, -final_component:] = c[:, -final_component:]
        reconstructed_image = reconstruct_image(pca, components, pca.mean_, mean, std, [1]+list(original_shape[1:]))

        ax2_[row,column].imshow(np.transpose(reconstructed_image[0], (1,2,0)),cmap="gray")
        ax2_[row,column].set_title(f"ratio {round(1-r,2)}")
        ax2_[row,column].axis('off')

    plt.show()


## CIFAR10 Reduced

In [ ]:
# Download and load training dataset
percentage_samples=0.2
folder = '/local/home/abizeul/data/cifar-10-batches-py'
trainset = torchvision.datasets.CIFAR10(root='/local/home/abizeul/data', train=True, download=True, transform=transform)
subset_indices = torch.randperm(len(trainset))[:int(percentage_samples*len(trainset))]
trainset = Subset(trainset, subset_indices)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=len(trainset), shuffle=False)

# Fetch the entire dataset in one go
data_iter = iter(trainloader)
images, labels = next(data_iter)

# Step 2: Convert the dataset to NumPy arrays
images_np = images.numpy()
labels_np = labels.numpy()

# Reshape the images to (num_samples, height * width * channels)
num_samples = images_np.shape[0]
original_shape = images_np.shape
images_flat = images_np.reshape(num_samples, -1)

# Standardize
mean, std   = np.mean(images_flat, axis=0), np.std(images_flat, axis=0)
images_flat = (images_flat - mean) / std

# Step 4: Perform PCA
pca = PCA()  # You can adjust the number of components
pca.fit(images_flat)

In [ ]:
fig = make_subplots(rows=1, cols=3, subplot_titles=("Eigenvalues", "Percentage explained variance","Cumulative sum of percentages"))

# First plot (left subplot)
fig.add_trace(go.Bar(
    x=np.arange(len(pca.explained_variance_)), 
    y=pca.explained_variance_,
    marker=dict(color='rgba(58, 71, 80, 0.6)', line=dict(color='rgba(58, 71, 80, 1.0)', width=1.5)),
    opacity=0.8
), row=1, col=1)

# Second plot (right subplot)
fig.add_trace(go.Bar(
    x=np.arange(len(pca.explained_variance_ratio_)), 
    y=pca.explained_variance_ratio_,  # Replace with a different dataset or transformation for a different plot
    marker=dict(color='rgba(120, 50, 80, 0.6)', line=dict(color='rgba(120, 50, 80, 1.0)', width=1.5)),
    opacity=0.8
), row=1, col=2)

fig.add_trace(go.Bar(
    x=np.arange(len(pca.explained_variance_ratio_)), 
    y=np.cumsum(pca.explained_variance_ratio_),  # Replace with a different dataset or transformation for a different plot
    marker=dict(color='rgba(120, 50, 80, 0.6)', line=dict(color='rgba(120, 50, 80, 1.0)', width=1.5)),
    opacity=0.8
), row=1, col=3)

# Update layout for aesthetics and specify the size
fig.update_layout(
    title_text='',
    xaxis=dict(
        title='',
        titlefont=dict(size=14),
        tickfont=dict(size=12),
        showgrid=True,  # Add grid lines for y-axis
        gridcolor='lightgrey',  # Color of grid lines
        gridwidth=0.5  # Width of grid lines
    ),
    yaxis=dict(
        title='',
        titlefont=dict(size=14),
        tickfont=dict(size=12),
        type='log',
        showgrid=True,  # Add grid lines for y-axis
        gridcolor='lightgrey',  # Color of grid lines
        gridwidth=0.5  # Width of grid lines
    ),
    plot_bgcolor='white',
    showlegend=False,
    margin=dict(l=40, r=40, t=40, b=40),
    width=1000,  # Specify width of the image
    height=600  # Specify height of the image
)

# Update x-axis and y-axis for the first subplot
fig.update_xaxes(title_text='Components', title_font=dict(size=14), tickfont=dict(size=12), 
                 showgrid=True, gridcolor='lightgrey', gridwidth=0.5, row=1, col=1)
fig.update_yaxes(title_text='Explained variance (Log Scale)', title_font=dict(size=14), tickfont=dict(size=12), 
                 type='log', showgrid=True, gridcolor='lightgrey', gridwidth=0.5, row=1, col=1)

# Update x-axis and y-axis for the second subplot
fig.update_xaxes(title_text='Components', title_font=dict(size=14), tickfont=dict(size=12), 
                 showgrid=True, gridcolor='lightgrey', gridwidth=0.5, row=1, col=2)
fig.update_yaxes(title_text='Explained variance (Log Scale)', title_font=dict(size=14), tickfont=dict(size=12), 
                 type='log', showgrid=True, gridcolor='lightgrey', gridwidth=0.5, row=1, col=2)

# Update x-axis and y-axis for the second subplot
fig.update_xaxes(title_text='Components', title_font=dict(size=14), tickfont=dict(size=12), 
                 showgrid=True, gridcolor='lightgrey', gridwidth=0.5, row=1, col=3)
fig.update_yaxes(title_text='Explained variance (Log Scale)', title_font=dict(size=14), tickfont=dict(size=12), 
                 type='log', showgrid=True, gridcolor='lightgrey', gridwidth=0.5, row=1, col=3)


# Save the figure as a PNG file
fig.write_image(f"{folder}/eigenvalues.png")

# Show the plot
fig.show()

In [ ]:
# Step 5: Visualize the result of pc filtering on one image
def reconstruct_image(pca, components, mean, std_mean, std_stdev, shape):
    image_reconstructed = np.dot(components, pca.components_) + mean
    image_reconstructed = (image_reconstructed * std_stdev) + std_mean
    image_reconstructed = image_reconstructed.reshape(shape)
    return image_reconstructed

ratios = [0.1,0.2,0.4,0.6,0.7,0.75,0.8,0.85,0.9,0.95,0.99]
nb_plots = len(ratios) + 1
nb_plots_row = 6
nb_components = min(images_flat.shape[0],images_flat.shape[1])
nb_plots_columns = int(np.ceil(nb_plots/nb_plots_row))

for j, img in enumerate(images_np[:2]):
    print(img.shape)
    fig, ax = plt.subplots(nb_plots_columns,nb_plots_row,figsize=(20,5))
    ax[0,0].imshow(np.transpose(img, (1,2,0)),cmap="gray")
    ax[0,0].set_title("Original Image")
    ax[0,0].axis('off')

    fig_, ax_ = plt.subplots(nb_plots_columns,nb_plots_row,figsize=(20,5))
    ax_[0,0].imshow(np.transpose(img, (1,2,0)),cmap="gray")
    ax_[0,0].set_title("Original Image")
    ax_[0,0].axis('off')

    fig2, ax2 = plt.subplots(nb_plots_columns,nb_plots_row,figsize=(20,5))
    ax2[0,0].imshow(np.transpose(img, (1,2,0)),cmap="gray")
    ax2[0,0].set_title("Original Image")
    ax2[0,0].axis('off')

    fig2_, ax2_ = plt.subplots(nb_plots_columns,nb_plots_row,figsize=(20,5))
    ax2_[0,0].imshow(np.transpose(img, (1,2,0)),cmap="gray")
    ax2_[0,0].set_title("Original Image")
    ax2_[0,0].axis('off')

    c=pca.transform(images_flat[j:j+1])
    cumsum = np.cumsum(pca.explained_variance_ratio_)

    for i, r in enumerate(ratios):
        row, column = (i+1)//nb_plots_row, (i+1)%nb_plots_row

        final_component = int(r*nb_components)
        components = np.zeros_like(c)
        components[:, :final_component] = c[:, :final_component]
        reconstructed_image = reconstruct_image(pca, components, pca.mean_, mean, std, [1]+list(original_shape[1:]))

        ax[row,column].imshow(np.transpose(reconstructed_image[0], (1,2,0)),cmap="gray")
        ax[row,column].set_title(f"ratio {r}")
        ax[row,column].axis('off')
        
        #######
        components = np.zeros_like(c)
        final_component = nb_components - final_component
        components[:, -final_component:] = c[:, -final_component:]
        reconstructed_image = reconstruct_image(pca, components, pca.mean_, mean, std, [1]+list(original_shape[1:]))

        ax_[row,column].imshow(np.transpose(reconstructed_image[0], (1,2,0)),cmap="gray")
        ax_[row,column].set_title(f"ratio {round(1-r,2)}")
        ax_[row,column].axis('off')

        #######
        final_component=int(np.argmin(np.abs(cumsum - r)))+1
        components = np.zeros_like(c)
        components[:, :final_component] = c[:, :final_component]
        reconstructed_image = reconstruct_image(pca, components, pca.mean_, mean, std, [1]+list(original_shape[1:]))

        ax2[row,column].imshow(np.transpose(reconstructed_image[0], (1,2,0)),cmap="gray")
        ax2[row,column].set_title(f"ratio {r}")
        ax2[row,column].axis('off')

        #######
        final_component = nb_components - final_component
        components = np.zeros_like(c)
        components[:, -final_component:] = c[:, -final_component:]
        reconstructed_image = reconstruct_image(pca, components, pca.mean_, mean, std, [1]+list(original_shape[1:]))

        ax2_[row,column].imshow(np.transpose(reconstructed_image[0], (1,2,0)),cmap="gray")
        ax2_[row,column].set_title(f"ratio {round(1-r,2)}")
        ax2_[row,column].axis('off')

    plt.show()

## TinyImageNet

In [2]:
folder = '/local/home/abizeul/data/tiny-imagenet-200'
trainset = torchvision.datasets.ImageFolder(folder, transform=transform)
trainloader =  torch.utils.data.DataLoader(trainset, batch_size=len(trainset), shuffle=False, num_workers=4)

# Fetch the entire dataset in one go
data_iter = iter(trainloader)
images, labels = next(data_iter)

# Step 2: Convert the dataset to NumPy arrays
images_np = images.numpy()
labels_np = labels.numpy()

# Reshape the images to (num_samples, height * width * channels)
num_samples = images_np.shape[0]
original_shape = images_np.shape
images_flat = images_np.reshape(num_samples, -1)

# Standardize
mean, std   = np.mean(images_flat, axis=0), np.std(images_flat, axis=0)
images_flat = (images_flat - mean) / std
nb_components = min(images_flat.shape[0],images_flat.shape[1])

# Step 4: Perform PCA
pca = PCA()  # You can adjust the number of components
pca.fit(images_flat)

PCA()

In [3]:
nb_components

12288

In [4]:
num_samples

120000

In [6]:
pca.components_.shape

(12288, 12288)

In [7]:
# save information
np.save(f'{folder}/pc_matrix.npy',pca.components_)
np.save(f'{folder}/eigenvalues.npy',pca.explained_variance_)
np.save(f'{folder}/eigenvalues_ratio.npy',pca.explained_variance_ratio_)

In [ ]:
# looking at eigenvalue decomposition 
fig = make_subplots(rows=1, cols=3, subplot_titles=("Eigenvalues", "Percentage explained variance","Cumulative sum of percentages"))

# First plot (left subplot)
fig.add_trace(go.Bar(
    x=np.arange(len(pca.explained_variance_)), 
    y=pca.explained_variance_,
    marker=dict(color='rgba(58, 71, 80, 0.6)', line=dict(color='rgba(58, 71, 80, 1.0)', width=1.5)),
    opacity=0.8
), row=1, col=1)

# Second plot (right subplot)
fig.add_trace(go.Bar(
    x=np.arange(len(pca.explained_variance_ratio_)), 
    y=pca.explained_variance_ratio_,  # Replace with a different dataset or transformation for a different plot
    marker=dict(color='rgba(120, 50, 80, 0.6)', line=dict(color='rgba(120, 50, 80, 1.0)', width=1.5)),
    opacity=0.8
), row=1, col=2)

fig.add_trace(go.Bar(
    x=np.arange(len(pca.explained_variance_ratio_)), 
    y=np.cumsum(pca.explained_variance_ratio_),  # Replace with a different dataset or transformation for a different plot
    marker=dict(color='rgba(120, 50, 80, 0.6)', line=dict(color='rgba(120, 50, 80, 1.0)', width=1.5)),
    opacity=0.8
), row=1, col=3)

# Update layout for aesthetics and specify the size
fig.update_layout(
    title_text='',
    xaxis=dict(
        title='',
        titlefont=dict(size=14),
        tickfont=dict(size=12),
        showgrid=True,  # Add grid lines for y-axis
        gridcolor='lightgrey',  # Color of grid lines
        gridwidth=0.5  # Width of grid lines
    ),
    yaxis=dict(
        title='',
        titlefont=dict(size=14),
        tickfont=dict(size=12),
        type='log',
        showgrid=True,  # Add grid lines for y-axis
        gridcolor='lightgrey',  # Color of grid lines
        gridwidth=0.5  # Width of grid lines
    ),
    plot_bgcolor='white',
    showlegend=False,
    margin=dict(l=40, r=40, t=40, b=40),
    width=1000,  # Specify width of the image
    height=600  # Specify height of the image
)

# Update x-axis and y-axis for the first subplot
fig.update_xaxes(title_text='Components', title_font=dict(size=14), tickfont=dict(size=12), 
                 showgrid=True, gridcolor='lightgrey', gridwidth=0.5, row=1, col=1)
fig.update_yaxes(title_text='Explained variance (Log Scale)', title_font=dict(size=14), tickfont=dict(size=12), 
                 type='log', showgrid=True, gridcolor='lightgrey', gridwidth=0.5, row=1, col=1)

# Update x-axis and y-axis for the second subplot
fig.update_xaxes(title_text='Components', title_font=dict(size=14), tickfont=dict(size=12), 
                 showgrid=True, gridcolor='lightgrey', gridwidth=0.5, row=1, col=2)
fig.update_yaxes(title_text='Explained variance (Log Scale)', title_font=dict(size=14), tickfont=dict(size=12), 
                 type='log', showgrid=True, gridcolor='lightgrey', gridwidth=0.5, row=1, col=2)

# Update x-axis and y-axis for the second subplot
fig.update_xaxes(title_text='Components', title_font=dict(size=14), tickfont=dict(size=12), 
                 showgrid=True, gridcolor='lightgrey', gridwidth=0.5, row=1, col=3)
fig.update_yaxes(title_text='Explained variance (Log Scale)', title_font=dict(size=14), tickfont=dict(size=12), 
                 type='log', showgrid=True, gridcolor='lightgrey', gridwidth=0.5, row=1, col=3)


# Save the figure as a PNG file
fig.write_image(f"{folder}/eigenvalues.png")

# Show the plot
fig.show()


In [ ]:
# Step 5: Visualize the result of pc filtering on one image
def reconstruct_image(pca, components, mean, std_mean, std_stdev, shape):
    image_reconstructed = np.dot(components, pca.components_) + mean
    image_reconstructed = (image_reconstructed * std_stdev) + std_mean
    image_reconstructed = image_reconstructed.reshape(shape)
    return image_reconstructed

ratios = [0.1,0.2,0.4,0.6,0.7,0.75,0.8,0.85,0.9,0.95,0.99]
nb_plots = len(ratios) + 1
nb_plots_row = 6
nb_components = min(images_flat.shape[0],images_flat.shape[1])
nb_plots_columns = int(np.ceil(nb_plots/nb_plots_row))

for j, img in enumerate(images_np[:2]):
    print(img.shape)
    fig, ax = plt.subplots(nb_plots_columns,nb_plots_row,figsize=(20,5))
    ax[0,0].imshow(np.transpose(img, (1,2,0)),cmap="gray")
    ax[0,0].set_title("Original Image")
    ax[0,0].axis('off')

    fig_, ax_ = plt.subplots(nb_plots_columns,nb_plots_row,figsize=(20,5))
    ax_[0,0].imshow(np.transpose(img, (1,2,0)),cmap="gray")
    ax_[0,0].set_title("Original Image")
    ax_[0,0].axis('off')

    fig2, ax2 = plt.subplots(nb_plots_columns,nb_plots_row,figsize=(20,5))
    ax2[0,0].imshow(np.transpose(img, (1,2,0)),cmap="gray")
    ax2[0,0].set_title("Original Image")
    ax2[0,0].axis('off')

    fig2_, ax2_ = plt.subplots(nb_plots_columns,nb_plots_row,figsize=(20,5))
    ax2_[0,0].imshow(np.transpose(img, (1,2,0)),cmap="gray")
    ax2_[0,0].set_title("Original Image")
    ax2_[0,0].axis('off')

    c=pca.transform(images_flat[j:j+1])
    cumsum = np.cumsum(pca.explained_variance_ratio_)

    for i, r in enumerate(ratios):
        row, column = (i+1)//nb_plots_row, (i+1)%nb_plots_row

        final_component = int(r*nb_components)
        components = np.zeros_like(c)
        components[:, :final_component] = c[:, :final_component]
        reconstructed_image = reconstruct_image(pca, components, pca.mean_, mean, std, [1]+list(original_shape[1:]))

        ax[row,column].imshow(np.transpose(reconstructed_image[0], (1,2,0)),cmap="gray")
        ax[row,column].set_title(f"ratio {r}")
        ax[row,column].axis('off')
        
        #######
        components = np.zeros_like(c)
        final_component = nb_components - final_component
        components[:, -final_component:] = c[:, -final_component:]
        reconstructed_image = reconstruct_image(pca, components, pca.mean_, mean, std, [1]+list(original_shape[1:]))

        ax_[row,column].imshow(np.transpose(reconstructed_image[0], (1,2,0)),cmap="gray")
        ax_[row,column].set_title(f"ratio {round(1-r,2)}")
        ax_[row,column].axis('off')

        #######
        final_component=int(np.argmin(np.abs(cumsum - r)))+1
        components = np.zeros_like(c)
        components[:, :final_component] = c[:, :final_component]
        reconstructed_image = reconstruct_image(pca, components, pca.mean_, mean, std, [1]+list(original_shape[1:]))

        ax2[row,column].imshow(np.transpose(reconstructed_image[0], (1,2,0)),cmap="gray")
        ax2[row,column].set_title(f"ratio {r}")
        ax2[row,column].axis('off')

        #######
        final_component = nb_components - final_component
        components = np.zeros_like(c)
        components[:, -final_component:] = c[:, -final_component:]
        reconstructed_image = reconstruct_image(pca, components, pca.mean_, mean, std, [1]+list(original_shape[1:]))

        ax2_[row,column].imshow(np.transpose(reconstructed_image[0], (1,2,0)),cmap="gray")
        ax2_[row,column].set_title(f"ratio {round(1-r,2)}")
        ax2_[row,column].axis('off')

    plt.show()

In [ ]:
from torchvision.utils import save_image
for index,img in enumerate(reconstructed_image):
    path, _ = trainset.samples[index]
    print(path)
    save_image(torch.tensor(img),path)

In [ ]:
# saving pcs 
np.save("/local/home/abizeul/reconstruction/pca_matrix/tiny_pcs.npy",pca.components_)
np.save("/local/home/abizeul/reconstruction/pca_matrix/tiny_pca_mean.npy",pca.mean_)

In [ ]:
fig, ax = plt.subplots(nb_plots_columns,nb_plots_row, figsize=(20, 10))

for i in range(1,final_component,granularity):
    
    # Optionally, you can inverse transform to see the approximation
    components = np.zeros_like(c)
    components[:, -i:] = c[:, -i:]
    # take only first sample
    components = components[:1]
    reconstructed_image = reconstruct_image(pca, components, pca.mean_, original_shape[1:])
    
    # Display the PCA reconstructed image
    index = (i)//granularity
    ax[index//nb_plots_row,index%nb_plots_row].imshow(np.transpose(reconstructed_image,(1,2,0)),)
    ax[index//nb_plots_row,index%nb_plots_row].set_title(f"{i}")
    ax[index//nb_plots_row,index%nb_plots_row].axis('off')

ax[nb_plots_columns-1,nb_plots_row-1].imshow(np.transpose(images_np[0], (1,2,0)),cmap="gray")
ax[nb_plots_columns-1,nb_plots_row-1].set_title("Original Image")
ax[nb_plots_columns-1,nb_plots_row-1].axis('off')

plt.show()

## DermaMNIST

In [ ]:
folder = '/local/home/abizeul/data/medmnist'
trainset = DermaMNIST(root=folder,split="train",download=True,size=224,transform=transform)
trainloader =  torch.utils.data.DataLoader(trainset, batch_size=len(trainset), shuffle=False, num_workers=4)

# Fetch the entire dataset in one go
data_iter = iter(trainloader)
images, labels = next(data_iter)

# Step 2: Convert the dataset to NumPy arrays
images_np = images.numpy()
labels_np = labels.numpy()

# Reshape the images to (num_samples, height * width * channels)
num_samples = images_np.shape[0]
original_shape = images_np.shape
images_flat = images_np.reshape(num_samples, -1)

# Standardize
mean, std   = np.mean(images_flat, axis=0), np.std(images_flat, axis=0)
images_flat = (images_flat - mean) / std

# Step 4: Perform PCA
pca = PCA()  # You can adjust the number of components
pca.fit(images_flat)

In [ ]:
# save information
np.save(f'{folder}/pc_matrix.npy',pca.components_)
np.save(f'{folder}/eigenvalues.npy',pca.explained_variance_)
np.save(f'{folder}/eigenvalues_ratio.npy',pca.explained_variance_ratio_)

In [ ]:
# looking at eigenvalue decomposition 
fig = make_subplots(rows=1, cols=3, subplot_titles=("Eigenvalues", "Percentage explained variance","Cumulative sum of percentages"))

# First plot (left subplot)
fig.add_trace(go.Bar(
    x=np.arange(len(pca.explained_variance_)), 
    y=pca.explained_variance_,
    marker=dict(color='rgba(58, 71, 80, 0.6)', line=dict(color='rgba(58, 71, 80, 1.0)', width=1.5)),
    opacity=0.8
), row=1, col=1)

# Second plot (right subplot)
fig.add_trace(go.Bar(
    x=np.arange(len(pca.explained_variance_ratio_)), 
    y=pca.explained_variance_ratio_,  # Replace with a different dataset or transformation for a different plot
    marker=dict(color='rgba(120, 50, 80, 0.6)', line=dict(color='rgba(120, 50, 80, 1.0)', width=1.5)),
    opacity=0.8
), row=1, col=2)

fig.add_trace(go.Bar(
    x=np.arange(len(pca.explained_variance_ratio_)), 
    y=np.cumsum(pca.explained_variance_ratio_),  # Replace with a different dataset or transformation for a different plot
    marker=dict(color='rgba(120, 50, 80, 0.6)', line=dict(color='rgba(120, 50, 80, 1.0)', width=1.5)),
    opacity=0.8
), row=1, col=3)

# Update layout for aesthetics and specify the size
fig.update_layout(
    title_text='',
    xaxis=dict(
        title='',
        titlefont=dict(size=14),
        tickfont=dict(size=12),
        showgrid=True,  # Add grid lines for y-axis
        gridcolor='lightgrey',  # Color of grid lines
        gridwidth=0.5  # Width of grid lines
    ),
    yaxis=dict(
        title='',
        titlefont=dict(size=14),
        tickfont=dict(size=12),
        type='log',
        showgrid=True,  # Add grid lines for y-axis
        gridcolor='lightgrey',  # Color of grid lines
        gridwidth=0.5  # Width of grid lines
    ),
    plot_bgcolor='white',
    showlegend=False,
    margin=dict(l=40, r=40, t=40, b=40),
    width=1000,  # Specify width of the image
    height=600  # Specify height of the image
)

# Update x-axis and y-axis for the first subplot
fig.update_xaxes(title_text='Components', title_font=dict(size=14), tickfont=dict(size=12), 
                 showgrid=True, gridcolor='lightgrey', gridwidth=0.5, row=1, col=1)
fig.update_yaxes(title_text='Explained variance (Log Scale)', title_font=dict(size=14), tickfont=dict(size=12), 
                 type='log', showgrid=True, gridcolor='lightgrey', gridwidth=0.5, row=1, col=1)

# Update x-axis and y-axis for the second subplot
fig.update_xaxes(title_text='Components', title_font=dict(size=14), tickfont=dict(size=12), 
                 showgrid=True, gridcolor='lightgrey', gridwidth=0.5, row=1, col=2)
fig.update_yaxes(title_text='Explained variance (Log Scale)', title_font=dict(size=14), tickfont=dict(size=12), 
                 type='log', showgrid=True, gridcolor='lightgrey', gridwidth=0.5, row=1, col=2)

# Update x-axis and y-axis for the second subplot
fig.update_xaxes(title_text='Components', title_font=dict(size=14), tickfont=dict(size=12), 
                 showgrid=True, gridcolor='lightgrey', gridwidth=0.5, row=1, col=3)
fig.update_yaxes(title_text='Explained variance (Log Scale)', title_font=dict(size=14), tickfont=dict(size=12), 
                 type='log', showgrid=True, gridcolor='lightgrey', gridwidth=0.5, row=1, col=3)


# Save the figure as a PNG file
fig.write_image(f"{folder}/eigenvalues.png")

# Show the plot
fig.show()

In [ ]:
# Step 5: Visualize the result of pc filtering on one image
def reconstruct_image(pca, components, mean, std_mean, std_stdev, shape):
    image_reconstructed = np.dot(components, pca.components_) + mean
    image_reconstructed = (image_reconstructed * std_stdev) + std_mean
    image_reconstructed = image_reconstructed.reshape(shape)
    return image_reconstructed

ratios = [0.1,0.2,0.4,0.6,0.7,0.75,0.8,0.85,0.9,0.95,0.99]
nb_plots = len(ratios) + 1
nb_plots_row = 6
nb_components = min(images_flat.shape[0],images_flat.shape[1])
nb_plots_columns = int(np.ceil(nb_plots/nb_plots_row))

for j, img in enumerate(images_np[:2]):
    print(img.shape)
    fig, ax = plt.subplots(nb_plots_columns,nb_plots_row,figsize=(20,5))
    ax[0,0].imshow(np.transpose(img, (1,2,0)),cmap="gray")
    ax[0,0].set_title("Original Image")
    ax[0,0].axis('off')

    fig_, ax_ = plt.subplots(nb_plots_columns,nb_plots_row,figsize=(20,5))
    ax_[0,0].imshow(np.transpose(img, (1,2,0)),cmap="gray")
    ax_[0,0].set_title("Original Image")
    ax_[0,0].axis('off')

    fig2, ax2 = plt.subplots(nb_plots_columns,nb_plots_row,figsize=(20,5))
    ax2[0,0].imshow(np.transpose(img, (1,2,0)),cmap="gray")
    ax2[0,0].set_title("Original Image")
    ax2[0,0].axis('off')

    fig2_, ax2_ = plt.subplots(nb_plots_columns,nb_plots_row,figsize=(20,5))
    ax2_[0,0].imshow(np.transpose(img, (1,2,0)),cmap="gray")
    ax2_[0,0].set_title("Original Image")
    ax2_[0,0].axis('off')

    c=pca.transform(images_flat[j:j+1])
    cumsum = np.cumsum(pca.explained_variance_ratio_)

    for i, r in enumerate(ratios):
        row, column = (i+1)//nb_plots_row, (i+1)%nb_plots_row

        final_component = int(r*nb_components)
        components = np.zeros_like(c)
        components[:, :final_component] = c[:, :final_component]
        reconstructed_image = reconstruct_image(pca, components, pca.mean_, mean, std, [1]+list(original_shape[1:]))

        ax[row,column].imshow(np.transpose(reconstructed_image[0], (1,2,0)),cmap="gray")
        ax[row,column].set_title(f"ratio {r}")
        ax[row,column].axis('off')
        
        #######
        components = np.zeros_like(c)
        final_component = nb_components - final_component
        components[:, -final_component:] = c[:, -final_component:]
        reconstructed_image = reconstruct_image(pca, components, pca.mean_, mean, std, [1]+list(original_shape[1:]))

        ax_[row,column].imshow(np.transpose(reconstructed_image[0], (1,2,0)),cmap="gray")
        ax_[row,column].set_title(f"ratio {round(1-r,2)}")
        ax_[row,column].axis('off')

        #######
        final_component=int(np.argmin(np.abs(cumsum - r)))+1
        components = np.zeros_like(c)
        components[:, :final_component] = c[:, :final_component]
        reconstructed_image = reconstruct_image(pca, components, pca.mean_, mean, std, [1]+list(original_shape[1:]))

        ax2[row,column].imshow(np.transpose(reconstructed_image[0], (1,2,0)),cmap="gray")
        ax2[row,column].set_title(f"ratio {r}")
        ax2[row,column].axis('off')

        #######
        final_component = nb_components - final_component
        components = np.zeros_like(c)
        components[:, -final_component:] = c[:, -final_component:]
        reconstructed_image = reconstruct_image(pca, components, pca.mean_, mean, std, [1]+list(original_shape[1:]))

        ax2_[row,column].imshow(np.transpose(reconstructed_image[0], (1,2,0)),cmap="gray")
        ax2_[row,column].set_title(f"ratio {round(1-r,2)}")
        ax2_[row,column].axis('off')

    plt.show()

## BloodMNIST

In [ ]:
folder = '/local/home/abizeul/data/medmnist'
trainset = BloodMNIST(root=folder,split="train",download=True,size=224,transform=transform)
trainloader =  torch.utils.data.DataLoader(trainset, batch_size=len(trainset), shuffle=False, num_workers=4)

# Fetch the entire dataset in one go
data_iter = iter(trainloader)
images, labels = next(data_iter)

# Step 2: Convert the dataset to NumPy arrays
images_np = images.numpy()
labels_np = labels.numpy()

# Reshape the images to (num_samples, height * width * channels)
num_samples = images_np.shape[0]
original_shape = images_np.shape
images_flat = images_np.reshape(num_samples, -1)

# Standardize
mean, std   = np.mean(images_flat, axis=0), np.std(images_flat, axis=0)
images_flat = (images_flat - mean) / std

# Step 4: Perform PCA
pca = PCA()  # You can adjust the number of components
pca.fit(images_flat)

In [ ]:
# save information
np.save(f'{folder}/pc_matrix.npy',pca.components_)
np.save(f'{folder}/eigenvalues.npy',pca.explained_variance_)
np.save(f'{folder}/eigenvalues_ratio.npy',pca.explained_variance_ratio_)

In [ ]:
# looking at eigenvalue decomposition 
fig = make_subplots(rows=1, cols=3, subplot_titles=("Eigenvalues", "Percentage explained variance","Cumulative sum of percentages"))

# First plot (left subplot)
fig.add_trace(go.Bar(
    x=np.arange(len(pca.explained_variance_)), 
    y=pca.explained_variance_,
    marker=dict(color='rgba(58, 71, 80, 0.6)', line=dict(color='rgba(58, 71, 80, 1.0)', width=1.5)),
    opacity=0.8
), row=1, col=1)

# Second plot (right subplot)
fig.add_trace(go.Bar(
    x=np.arange(len(pca.explained_variance_ratio_)), 
    y=pca.explained_variance_ratio_,  # Replace with a different dataset or transformation for a different plot
    marker=dict(color='rgba(120, 50, 80, 0.6)', line=dict(color='rgba(120, 50, 80, 1.0)', width=1.5)),
    opacity=0.8
), row=1, col=2)

fig.add_trace(go.Bar(
    x=np.arange(len(pca.explained_variance_ratio_)), 
    y=np.cumsum(pca.explained_variance_ratio_),  # Replace with a different dataset or transformation for a different plot
    marker=dict(color='rgba(120, 50, 80, 0.6)', line=dict(color='rgba(120, 50, 80, 1.0)', width=1.5)),
    opacity=0.8
), row=1, col=3)

# Update layout for aesthetics and specify the size
fig.update_layout(
    title_text='',
    xaxis=dict(
        title='',
        titlefont=dict(size=14),
        tickfont=dict(size=12),
        showgrid=True,  # Add grid lines for y-axis
        gridcolor='lightgrey',  # Color of grid lines
        gridwidth=0.5  # Width of grid lines
    ),
    yaxis=dict(
        title='',
        titlefont=dict(size=14),
        tickfont=dict(size=12),
        type='log',
        showgrid=True,  # Add grid lines for y-axis
        gridcolor='lightgrey',  # Color of grid lines
        gridwidth=0.5  # Width of grid lines
    ),
    plot_bgcolor='white',
    showlegend=False,
    margin=dict(l=40, r=40, t=40, b=40),
    width=1000,  # Specify width of the image
    height=600  # Specify height of the image
)

# Update x-axis and y-axis for the first subplot
fig.update_xaxes(title_text='Components', title_font=dict(size=14), tickfont=dict(size=12), 
                 showgrid=True, gridcolor='lightgrey', gridwidth=0.5, row=1, col=1)
fig.update_yaxes(title_text='Explained variance (Log Scale)', title_font=dict(size=14), tickfont=dict(size=12), 
                 type='log', showgrid=True, gridcolor='lightgrey', gridwidth=0.5, row=1, col=1)

# Update x-axis and y-axis for the second subplot
fig.update_xaxes(title_text='Components', title_font=dict(size=14), tickfont=dict(size=12), 
                 showgrid=True, gridcolor='lightgrey', gridwidth=0.5, row=1, col=2)
fig.update_yaxes(title_text='Explained variance (Log Scale)', title_font=dict(size=14), tickfont=dict(size=12), 
                 type='log', showgrid=True, gridcolor='lightgrey', gridwidth=0.5, row=1, col=2)

# Update x-axis and y-axis for the second subplot
fig.update_xaxes(title_text='Components', title_font=dict(size=14), tickfont=dict(size=12), 
                 showgrid=True, gridcolor='lightgrey', gridwidth=0.5, row=1, col=3)
fig.update_yaxes(title_text='Explained variance (Log Scale)', title_font=dict(size=14), tickfont=dict(size=12), 
                 type='log', showgrid=True, gridcolor='lightgrey', gridwidth=0.5, row=1, col=3)


# Save the figure as a PNG file
fig.write_image(f"{folder}/eigenvalues.png")

# Show the plot
fig.show()

In [ ]:
# Step 5: Visualize the result of pc filtering on one image
def reconstruct_image(pca, components, mean, std_mean, std_stdev, shape):
    image_reconstructed = np.dot(components, pca.components_) + mean
    image_reconstructed = (image_reconstructed * std_stdev) + std_mean
    image_reconstructed = image_reconstructed.reshape(shape)
    return image_reconstructed

ratios = [0.1,0.2,0.4,0.6,0.7,0.75,0.8,0.85,0.9,0.95,0.99]
nb_plots = len(ratios) + 1
nb_plots_row = 6
nb_components = min(images_flat.shape[0],images_flat.shape[1])
nb_plots_columns = int(np.ceil(nb_plots/nb_plots_row))

for j, img in enumerate(images_np[:2]):
    print(img.shape)
    fig, ax = plt.subplots(nb_plots_columns,nb_plots_row,figsize=(20,5))
    ax[0,0].imshow(np.transpose(img, (1,2,0)),cmap="gray")
    ax[0,0].set_title("Original Image")
    ax[0,0].axis('off')

    fig_, ax_ = plt.subplots(nb_plots_columns,nb_plots_row,figsize=(20,5))
    ax_[0,0].imshow(np.transpose(img, (1,2,0)),cmap="gray")
    ax_[0,0].set_title("Original Image")
    ax_[0,0].axis('off')

    fig2, ax2 = plt.subplots(nb_plots_columns,nb_plots_row,figsize=(20,5))
    ax2[0,0].imshow(np.transpose(img, (1,2,0)),cmap="gray")
    ax2[0,0].set_title("Original Image")
    ax2[0,0].axis('off')

    fig2_, ax2_ = plt.subplots(nb_plots_columns,nb_plots_row,figsize=(20,5))
    ax2_[0,0].imshow(np.transpose(img, (1,2,0)),cmap="gray")
    ax2_[0,0].set_title("Original Image")
    ax2_[0,0].axis('off')

    c=pca.transform(images_flat[j:j+1])
    cumsum = np.cumsum(pca.explained_variance_ratio_)

    for i, r in enumerate(ratios):
        row, column = (i+1)//nb_plots_row, (i+1)%nb_plots_row

        final_component = int(r*nb_components)
        components = np.zeros_like(c)
        components[:, :final_component] = c[:, :final_component]
        reconstructed_image = reconstruct_image(pca, components, pca.mean_, mean, std, [1]+list(original_shape[1:]))

        ax[row,column].imshow(np.transpose(reconstructed_image[0], (1,2,0)),cmap="gray")
        ax[row,column].set_title(f"ratio {r}")
        ax[row,column].axis('off')
        
        #######
        components = np.zeros_like(c)
        final_component = nb_components - final_component
        components[:, -final_component:] = c[:, -final_component:]
        reconstructed_image = reconstruct_image(pca, components, pca.mean_, mean, std, [1]+list(original_shape[1:]))

        ax_[row,column].imshow(np.transpose(reconstructed_image[0], (1,2,0)),cmap="gray")
        ax_[row,column].set_title(f"ratio {round(1-r,2)}")
        ax_[row,column].axis('off')

        #######
        final_component=int(np.argmin(np.abs(cumsum - r)))+1
        components = np.zeros_like(c)
        components[:, :final_component] = c[:, :final_component]
        reconstructed_image = reconstruct_image(pca, components, pca.mean_, mean, std, [1]+list(original_shape[1:]))

        ax2[row,column].imshow(np.transpose(reconstructed_image[0], (1,2,0)),cmap="gray")
        ax2[row,column].set_title(f"ratio {r}")
        ax2[row,column].axis('off')

        #######
        final_component = nb_components - final_component
        components = np.zeros_like(c)
        components[:, -final_component:] = c[:, -final_component:]
        reconstructed_image = reconstruct_image(pca, components, pca.mean_, mean, std, [1]+list(original_shape[1:]))

        ax2_[row,column].imshow(np.transpose(reconstructed_image[0], (1,2,0)),cmap="gray")
        ax2_[row,column].set_title(f"ratio {round(1-r,2)}")
        ax2_[row,column].axis('off')

    plt.show()

## PathMNIST

In [ ]:
folder = '/local/home/abizeul/data/medmnist'
trainset = PathMNIST(root=folder,split="train",download=True,size=224,transform=transform)
trainloader =  torch.utils.data.DataLoader(trainset, batch_size=len(trainset), shuffle=False, num_workers=4)

# Fetch the entire dataset in one go
data_iter = iter(trainloader)
images, labels = next(data_iter)

# Step 2: Convert the dataset to NumPy arrays
images_np = images.numpy()
labels_np = labels.numpy()

# Reshape the images to (num_samples, height * width * channels)
num_samples = images_np.shape[0]
original_shape = images_np.shape
images_flat = images_np.reshape(num_samples, -1)

# Standardize
mean, std   = np.mean(images_flat, axis=0), np.std(images_flat, axis=0)
images_flat = (images_flat - mean) / std

# Step 4: Perform PCA
pca = PCA()  # You can adjust the number of components
pca.fit(images_flat)

In [ ]:
# looking at eigenvalue decomposition 
fig = make_subplots(rows=1, cols=3, subplot_titles=("Eigenvalues", "Percentage explained variance","Cumulative sum of percentages"))

# First plot (left subplot)
fig.add_trace(go.Bar(
    x=np.arange(len(pca.explained_variance_)), 
    y=pca.explained_variance_,
    marker=dict(color='rgba(58, 71, 80, 0.6)', line=dict(color='rgba(58, 71, 80, 1.0)', width=1.5)),
    opacity=0.8
), row=1, col=1)

# Second plot (right subplot)
fig.add_trace(go.Bar(
    x=np.arange(len(pca.explained_variance_ratio_)), 
    y=pca.explained_variance_ratio_,  # Replace with a different dataset or transformation for a different plot
    marker=dict(color='rgba(120, 50, 80, 0.6)', line=dict(color='rgba(120, 50, 80, 1.0)', width=1.5)),
    opacity=0.8
), row=1, col=2)

fig.add_trace(go.Bar(
    x=np.arange(len(pca.explained_variance_ratio_)), 
    y=np.cumsum(pca.explained_variance_ratio_),  # Replace with a different dataset or transformation for a different plot
    marker=dict(color='rgba(120, 50, 80, 0.6)', line=dict(color='rgba(120, 50, 80, 1.0)', width=1.5)),
    opacity=0.8
), row=1, col=3)

# Update layout for aesthetics and specify the size
fig.update_layout(
    title_text='',
    xaxis=dict(
        title='',
        titlefont=dict(size=14),
        tickfont=dict(size=12),
        showgrid=True,  # Add grid lines for y-axis
        gridcolor='lightgrey',  # Color of grid lines
        gridwidth=0.5  # Width of grid lines
    ),
    yaxis=dict(
        title='',
        titlefont=dict(size=14),
        tickfont=dict(size=12),
        type='log',
        showgrid=True,  # Add grid lines for y-axis
        gridcolor='lightgrey',  # Color of grid lines
        gridwidth=0.5  # Width of grid lines
    ),
    plot_bgcolor='white',
    showlegend=False,
    margin=dict(l=40, r=40, t=40, b=40),
    width=1000,  # Specify width of the image
    height=600  # Specify height of the image
)

# Update x-axis and y-axis for the first subplot
fig.update_xaxes(title_text='Components', title_font=dict(size=14), tickfont=dict(size=12), 
                 showgrid=True, gridcolor='lightgrey', gridwidth=0.5, row=1, col=1)
fig.update_yaxes(title_text='Explained variance (Log Scale)', title_font=dict(size=14), tickfont=dict(size=12), 
                 type='log', showgrid=True, gridcolor='lightgrey', gridwidth=0.5, row=1, col=1)

# Update x-axis and y-axis for the second subplot
fig.update_xaxes(title_text='Components', title_font=dict(size=14), tickfont=dict(size=12), 
                 showgrid=True, gridcolor='lightgrey', gridwidth=0.5, row=1, col=2)
fig.update_yaxes(title_text='Explained variance (Log Scale)', title_font=dict(size=14), tickfont=dict(size=12), 
                 type='log', showgrid=True, gridcolor='lightgrey', gridwidth=0.5, row=1, col=2)

# Update x-axis and y-axis for the second subplot
fig.update_xaxes(title_text='Components', title_font=dict(size=14), tickfont=dict(size=12), 
                 showgrid=True, gridcolor='lightgrey', gridwidth=0.5, row=1, col=3)
fig.update_yaxes(title_text='Explained variance (Log Scale)', title_font=dict(size=14), tickfont=dict(size=12), 
                 type='log', showgrid=True, gridcolor='lightgrey', gridwidth=0.5, row=1, col=3)


# Save the figure as a PNG file
fig.write_image(f"{folder}/eigenvalues.png")

# Show the plot
fig.show()

In [ ]:
# Step 5: Visualize the result of pc filtering on one image
def reconstruct_image(pca, components, mean, std_mean, std_stdev, shape):
    image_reconstructed = np.dot(components, pca.components_) + mean
    image_reconstructed = (image_reconstructed * std_stdev) + std_mean
    image_reconstructed = image_reconstructed.reshape(shape)
    return image_reconstructed

ratios = [0.1,0.2,0.4,0.6,0.7,0.75,0.8,0.85,0.9,0.95,0.99]
nb_plots = len(ratios) + 1
nb_plots_row = 6
nb_components = min(images_flat.shape[0],images_flat.shape[1])
nb_plots_columns = int(np.ceil(nb_plots/nb_plots_row))

for j, img in enumerate(images_np[:2]):
    print(img.shape)
    fig, ax = plt.subplots(nb_plots_columns,nb_plots_row,figsize=(20,5))
    ax[0,0].imshow(np.transpose(img, (1,2,0)),cmap="gray")
    ax[0,0].set_title("Original Image")
    ax[0,0].axis('off')

    fig_, ax_ = plt.subplots(nb_plots_columns,nb_plots_row,figsize=(20,5))
    ax_[0,0].imshow(np.transpose(img, (1,2,0)),cmap="gray")
    ax_[0,0].set_title("Original Image")
    ax_[0,0].axis('off')

    fig2, ax2 = plt.subplots(nb_plots_columns,nb_plots_row,figsize=(20,5))
    ax2[0,0].imshow(np.transpose(img, (1,2,0)),cmap="gray")
    ax2[0,0].set_title("Original Image")
    ax2[0,0].axis('off')

    fig2_, ax2_ = plt.subplots(nb_plots_columns,nb_plots_row,figsize=(20,5))
    ax2_[0,0].imshow(np.transpose(img, (1,2,0)),cmap="gray")
    ax2_[0,0].set_title("Original Image")
    ax2_[0,0].axis('off')

    c=pca.transform(images_flat[j:j+1])
    cumsum = np.cumsum(pca.explained_variance_ratio_)

    for i, r in enumerate(ratios):
        row, column = (i+1)//nb_plots_row, (i+1)%nb_plots_row

        final_component = int(r*nb_components)
        components = np.zeros_like(c)
        components[:, :final_component] = c[:, :final_component]
        reconstructed_image = reconstruct_image(pca, components, pca.mean_, mean, std, [1]+list(original_shape[1:]))

        ax[row,column].imshow(np.transpose(reconstructed_image[0], (1,2,0)),cmap="gray")
        ax[row,column].set_title(f"ratio {r}")
        ax[row,column].axis('off')
        
        #######
        components = np.zeros_like(c)
        final_component = nb_components - final_component
        components[:, -final_component:] = c[:, -final_component:]
        reconstructed_image = reconstruct_image(pca, components, pca.mean_, mean, std, [1]+list(original_shape[1:]))

        ax_[row,column].imshow(np.transpose(reconstructed_image[0], (1,2,0)),cmap="gray")
        ax_[row,column].set_title(f"ratio {round(1-r,2)}")
        ax_[row,column].axis('off')

        #######
        final_component=int(np.argmin(np.abs(cumsum - r)))+1
        components = np.zeros_like(c)
        components[:, :final_component] = c[:, :final_component]
        reconstructed_image = reconstruct_image(pca, components, pca.mean_, mean, std, [1]+list(original_shape[1:]))

        ax2[row,column].imshow(np.transpose(reconstructed_image[0], (1,2,0)),cmap="gray")
        ax2[row,column].set_title(f"ratio {r}")
        ax2[row,column].axis('off')

        #######
        final_component = nb_components - final_component
        components = np.zeros_like(c)
        components[:, -final_component:] = c[:, -final_component:]
        reconstructed_image = reconstruct_image(pca, components, pca.mean_, mean, std, [1]+list(original_shape[1:]))

        ax2_[row,column].imshow(np.transpose(reconstructed_image[0], (1,2,0)),cmap="gray")
        ax2_[row,column].set_title(f"ratio {round(1-r,2)}")
        ax2_[row,column].axis('off')

    plt.show()